In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests

In [2]:
import random
from datetime import datetime

start_time = datetime.now()

In [3]:
def frist_name(browser, num):
    try:
        name = browser.find_element(By.XPATH, f'//*[@id="content"]/div[3]/ul/li[{num}]/div/div[1]/strong/a[1]/strong').text
    except:
        name = 'Nan'

    return name


def frist_rating(browser, num):
    try:
        rating = browser.find_element(By.CSS_SELECTOR, f'#content > div.list_wrap > ul > li:nth-child({num}) > div.info_area > div.related.v2 > div > span:nth-child(2) > div > a > span.u_likeit_text.num._count').text
    except:
        rating = browser.find_element(By.CSS_SELECTOR, f'#content > div.list_wrap > ul > li:nth-child({num}) > div.info_area > div.related.v2 > div > span:nth-child(2) > div > a > span.u_likeit_text._count').text     

    return rating


def frist_url(browser, num):
    try:
        url = browser.find_element(By.XPATH, f'//*[@id="content"]/div[3]/ul/li[{num}]/div/div[1]/strong/a[1]').get_attribute('href')
    except:
        url = 'Nan'
    
    return url


def page_check(browser):
    if browser.find_element(By.XPATH, '//*[@id="paginate"]/span[2]/a').get_attribute('href')[-1] == '#':
        return True

In [4]:
r = open('pill_list.txt', 'r')
pill_list = [i.rstrip() for i in r.readlines()]

random.shuffle(pill_list)

In [5]:
pill_list.__len__()

2031

In [6]:
df_url = pd.DataFrame(columns=['이름', 'rating', 'url'])
df_url_idx = 0

In [7]:
url = f'https://terms.naver.com/medicineSearch.naver?mode=nameSearch&query=' #{}&page=1

In [8]:
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [9]:
for pill in pill_list[:]:
    page = 1
    while True:
        browser.get(url + pill + '&page=' + str(page))
        time.sleep(1.5) # 인터넷 상태에 따라서 조정하여 사용. 너무 짧을 경우 rating이 수집되지 않는 경우가 있음.
        # browser.implicitly_wait(1.5)
        browser.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

        for i in range(1, 10+1):
            try:
                pill_rating = frist_rating(browser, i) 
            except:
                break

            pill_name = frist_name(browser, i)
            if pill_name in df_url['이름']: continue

            pill_url = frist_url(browser, i)


            df_url.loc[df_url_idx] = [pill_name, pill_rating, pill_url]
            # print([pill_name, pill_rating, pill_url])
            df_url_idx += 1
            
        page += 1

        if page_check: break


In [23]:
df_url['rating'] = df_url['rating'].apply(lambda x: 0 if x == '공감' else x)
df_url.to_csv('url_df.csv', encoding='utf-8-sig', index=False)
df_url['rating'] = df_url['rating'].astype(int)

In [24]:
target_url = df_url[(df_url['rating']>=5)].reset_index(drop=True)
target_url

,이름,rating,url
0,뉴본정50단위(칼리디노게나제) [Newborn Tab. 50unit],27,https://terms.naver.com/entry.naver?docId=2135...
1,크레인정50단위(칼리디노게나제) [Krein Tab.],29,https://terms.naver.com/entry.naver?docId=2141...
2,카레인정50단위(칼리디노게나제) [Kalein Tab. 50IU],14,https://terms.naver.com/entry.naver?docId=2164...
3,튜비스투정150/300mg [TUBES2 Tablet 150/300mg],10,https://terms.naver.com/entry.naver?docId=2164...
4,튜비스정 [TUBES Tablet],8,https://terms.naver.com/entry.naver?docId=2128...
...,...,...,...
562,콜락연질캡슐 [Collag soft cap.],12,https://terms.naver.com/entry.naver?docId=2131...
563,젠빅연질캡슐 [Genbig Soft Cap.],13,https://terms.naver.com/entry.naver?docId=2155...
564,쎄투연질캡슐 [Cettu Soft Capsule],16,https://terms.naver.com/entry.naver?docId=2134...
565,레비아정(날트렉손염산염) [Revia Tab. (Naltrexone Hcl)],13,https://terms.naver.com/entry.naver?docId=2141...


In [25]:
final_df = pd.DataFrame(columns=["이름", "성분정보", "효능효과", "용법용량", "저장방법", "사용기간", "사용상주의사항", 'url'])
final_df_idx = 0

In [26]:
for i in range(len(target_url)):
    res = requests.get(target_url.iloc[i,2])
    res.raise_for_status()                
    soup = BeautifulSoup(res.text, "lxml")
    temp = soup.find_all('p', {'class': 'txt'})
    temp = list(temp)

    try:
        final_df.loc[final_df_idx] = [target_url.iloc[i,0], 
                                    temp[0], 
                                    temp[1],
                                    temp[2],
                                    temp[3],
                                    temp[4],
                                    temp[5],
                                    target_url.iloc[i,2]]
        final_df_idx += 1
    except:
        final_df.loc[final_df_idx] = [target_url.iloc[i,0], 
                                    "Nan",
                                    temp[0], 
                                    temp[1],
                                    temp[2],
                                    temp[3],
                                    temp[4],
                                    # temp[5],
                                    target_url.iloc[i,2]]
        final_df_idx += 1
        print(target_url.iloc[i,2])

https://terms.naver.com/entry.naver?docId=2128400&cid=51000&categoryId=51000
https://terms.naver.com/entry.naver?docId=2128400&cid=51000&categoryId=51000


In [ ]:
def pre_word(x):
    return str(x).replace('<p class="txt">',"").replace('<b>',"").replace('</b>',"").replace("</p>","").rstrip().replace("<br/>","\n").replace("&gt;",">").replace("&lt;","<")

for c in final_df.columns.tolist():
    final_df[c] = final_df[c].apply(lambda x: pre_word(x).lstrip().rstrip())

In [ ]:
final_df.to_csv('final_df.csv', index=False, encoding='utf-8-sig')

In [ ]:
end_time = datetime.now()

print(end_time-start_time)

0:19:32.305673


In [132]:
# 표
res = requests.get('https://terms.naver.com/entry.naver?docId=2130875&cid=51000&categoryId=51000')
res.raise_for_status()                
soup = BeautifulSoup(res.text, "lxml")
temp = soup.find_all('p', {'class': 'txt'})
temp = list(temp)

In [ ]:
temp.find_a

In [133]:
temp

/html/body/div[1]/div[3]/div[1]/div[2]/div[2]/text()[1]
/html/body/div[1]/div[3]/div[1]/div[2]/div[2]/text()[2]
/html/body/div[1]/div[3]/div[1]/div[2]/div[2]/text()[139]

[<p class="txt"><b>이 약 1정 (260mg) 중</b><br/>다파글리플로진프로판디올수화물 12.3mg</p>,
 <p class="txt">1. 제 2형 당뇨병: 이 약은 제 2형 당뇨병 환자의 혈당 조절을 향상시키기 위해 식사요법 및 운동요법의 보조제로 투여한다.<br/>- 단독요법<br/>- 병용요법<br/>혈당이 충분히 조절되지 않는 제2형 당뇨병 환자 중 심혈관계 질환이 확인되었거나 심혈관계 위험인자가 있는 환자에서 심혈관계 사건 발생에 대한 영향은 '사용상의 주의사항, 11. 전문가를 위한 정보 3) 임상시험 정보' 항을 참고한다.<br/>2. 만성 심부전 : 만성 심부전(NYHA class II-IV) 환자에서 심혈관 질환으로 인한 사망, 심부전으로 인한 입원 및 심부전으로 인한 긴급 병원 방문 위험성 감소<br/>이 약은 다른 심부전 표준요법과 병용하여 투여한다.<br/>3. 만성 신장병 : 만성 신장병 환자에서 추정 사구체 여과율 [estimated glomerular filtration rate (eGFR)]의 지속적인 감소, 말기 신장병에 도달, 심혈관 질환으로 인한 사망 및 신장 관련 사망 위험성 감소<br/>이 약은 다른 신장병 표준요법과 병용하여 투여한다.</p>,
 <p class="txt">제2형 당뇨병<br/>단독 요법 및 추가 병용 요법<br/>이 약의 권장 용량은 단독 요법 및 인슐린 등 다른 혈당 강하제와의 추가 병용 요법에 대하여 1일 1회 10mg이다. 이 약을 인슐린 또는 설포닐우레아와 같은 인슐린 분비 촉진제와 병용하여 사용하는 경우, 저혈당의 위험을 줄이기 위해 더 낮은 용량의 인슐린 또는 인슐린 분비 촉진제를 고려할 수 있다.<br/>초기 병용요법<br/>이전 당뇨병 약물치료를 받은 경험이 없는 경우 메트포르민과 병용투여 시, 이 약의 초기 권장용량은 1일 1회 5mg 또는 1일 1회 10mg이다.<br/>만성 심부전 및 만성 신장병<br/>이 약의 권장용량은 1일 1회 10 mg이다.<br/>특수

In [ ]:
w = open('temp_str.txt', 'w', encoding='utf-8')
w.write(t)
w.close()

In [31]:
box = soup.find_all('div', {'class': 'box_tbl'})
box = list(box)


In [130]:
temp = []
for i in box:
    temp.append(str(i))


t = ''.join(temp)

In [125]:
t

'<div class="box_tbl"><table border="1" cellspacing="0" class="tb_style2">\n<caption></caption>\n<tbody>\n<tr>\n<td style="border-color:black; vertical-align:top; width:65.15pt"><p>\n</p><p style="margin-left:0cm; margin-right:0cm">기관계</p>\n</td>\n<td style="border-color:black black black #000000; vertical-align:top; width:68pt"><p>\n</p><p style="margin-left:0cm; margin-right:0cm">매우 흔하게</p>\n</td>\n<td style="border-color:black black black #000000; vertical-align:top; width:66.5pt"><p>\n</p><p style="margin-left:0cm; margin-right:0cm">흔하게<sup>*</sup></p>\n</td>\n<td style="border-color:black black black #000000; vertical-align:top; width:65.95pt"><p>\n</p><p style="margin-left:0cm; margin-right:0cm">흔하지 않게<sup>**</sup></p>\n</td>\n<td style="border-color:black black black #000000; vertical-align:top; width:63.55pt"><p>\n</p><p style="margin-left:0cm; margin-right:0cm">드물게</p>\n</td>\n<td style="border-color:black black black #000000; vertical-align:top; width:61.65pt"><p>\n</p><p sty

In [127]:
t = t[:t.index("<table")] + t[t.index("</table>"):]
t = t.replace("</table>", "", 1)

In [128]:
t

'<div class="box_tbl"></div><div class="box_tbl"></div><div class="box_tbl"><table border="1" cellspacing="0" class="tb_style2">\n<caption></caption>\n<tbody>\n<tr>\n<td style="width:103px"><p>\xa0</p></td>\n<td style="width:103px"><p>\xa0</p></td>\n<td style="width:235px"><p>\n</p><p style="text-align:center">인과관계와 상관없는</p>\n<p style="text-align:center">예상하지 못한 이상사례</p>\n<p style="text-align:center">15.73%(476/3,027명, 624건)</p>\n</td>\n<td style="width:203px"><p>\n</p><p style="text-align:center">인과관계를 배제할 수 없는 예상하지 못한 약물이상반응</p>\n<p style="text-align:center">3.67%(111/3,027명, 129건)</p>\n</td>\n</tr>\n<tr>\n<td rowspan="22" style="width:103px"><p>드물게(0.01～0.1%미만)</p></td>\n<td style="width:103px"><p>위장관계 장애</p></td>\n<td style="width:235px"><p>만성위염, 장게실, 위창자내공기참, 잦은장운동, 위장염, 대장용종, 장천공, 명치불편, 기능성위장장애, 미란성위염, 잇몸장애, 잇몸통증, 치질, 열공탈장, 과민성대장증후군, 흑색변, 급성췌장염, 만성췌장염</p></td>\n<td style="width:203px"><p>위염, 위장장애, 복부불편함, 복통, 식도염, 하복부통, 잦은장운동</p></td>\n</tr>\n<tr>\n<td style="width:103px"><p>비뇨기계 

In [129]:
soup = BeautifulSoup(t, "html.parser")
w = open('temp_str_1.txt', 'w', encoding='utf-8')
w.write(soup.text)
w.close()